In [1]:
import numpy as np
import tensorflow as tf
import tensorflow.contrib.layers as layers
import matplotlib.pyplot as plt
from sklearn.feature_selection import mutual_info_regression
import math 

In [2]:
# data
d=100
L=50
H=50
W=50
eta_X=np.random.normal(0, 2, (d, d))
eta_N=np.random.normal(4, 2, (d, d))
eta_Y=eta_X+eta_N
meu_X=np.zeros(d)
meu_N=np.ones(d)
def func(x):
    return x

def gen_X():
    return np.random.multivariate_normal( mean=meu_X,
                                  cov=eta_X,
                                  size = (L,W,H))
def noise():
    return np.random.multivariate_normal( mean=meu_N,
                                  cov=eta_N,
                                 size = (L,W,H))

def gen_Y(X,N):
    return func(X)+func(N)
data_size = 20000
X=gen_X()
N=noise()
y=gen_Y(X,N)
mi = 0.5*math.log(np.linalg.det(eta_Y)/np.linalg.det(eta_X),2)
print(mi)

C:\Users\ahmedadel\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:17: RuntimeWarning: covariance is not symmetric positive-semidefinite.
C:\Users\ahmedadel\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:21: RuntimeWarning: covariance is not symmetric positive-semidefinite.


28.318787370883925


In [3]:
print(X.shape)
print(y.shape)

(50, 50, 50, 100)
(50, 50, 50, 100)


In [4]:
def ma(array, policy,window_size=None, beta=None):
    x=np.zeros(len(array))
    if policy=='window':
        for i in range(0, len(array)):
            if i<window_size:
                x[i]= np.mean(array[0: i])
            else:
                x[i]= np.mean(array[i-window_size: i])
    elif policy=='weighted':
        x[0]=array[0]
        for i in range(1, len(array)):
               x[i]=x[i-1]*(1-beta)+beta*array[i]
    return x


In [ ]:
H=20
n_epochs = 2000
data_size = 20000

def MINE(x_in, y_in):
    
    # shuffle and concatenate
    y_shuffle = tf.random_shuffle(y_in)
    x_conc = tf.concat([x_in, x_in], axis=0)
    y_conc = tf.concat([y_in, y_shuffle], axis=0)
    
    # propagate the forward pass
    layerx = layers.linear(x_conc, H)
    layery = layers.linear(y_conc, H)
    layer2 = tf.nn.relu(layerx + layery)
    output = layers.linear(layer2, 1)
    
    # split in T_xy and T_x_y predictions
    N_samples = tf.shape(x_in)[0]
    T_xy = output[:N_samples]
    T_x_y = output[N_samples:]
    # compute the negative loss (maximise loss == minimise -loss)
    neg_loss = -(tf.reduce_mean(T_xy, axis=0) - tf.math.log(tf.reduce_mean(tf.math.exp(T_x_y))))
    opt = tf.train.AdamOptimizer(learning_rate=0.01).minimize(neg_loss)

    return neg_loss, opt

# prepare the placeholders for inputs
x_in = tf.placeholder(tf.float32, [None, 1], name='x_in')
y_in = tf.placeholder(tf.float32, [None, 1], name='y_in')

# make the loss and optimisation graphs
neg_loss, opt = MINE(x_in, y_in)

# start the session
sess = tf.InteractiveSession()
sess.run(tf.global_variables_initializer())

# train
MIs = []
for epoch in range(n_epochs):
    
    # generate the data
    x_sample=gen_X()
    noise_sample=noise()
    Y=gen_Y(x_sample,noise_sample)
    X_1D=X.reshape(-1,1)
    Y_1D=Y.reshape(-1,1)
    print (epoch, end="\r")

    # perform the training step
    feed_dict = {x_in:X_1D, y_in:X_1D}
    _, neg_l = sess.run([opt, neg_loss], feed_dict=feed_dict)
    
    # save the loss
    MIs.append(-neg_l)

mv_av=ma(MIs, policy='weighted', beta=0.01)



fig, ax = plt.subplots()
ax.plot([0, len(MIs)], [mi,mi], label='True Mutual Information')
ax.plot(range(len(mv_av)), mv_av, label='Moving Average of MINE (window policy)')

ax.set_xlabel('training steps')
ax.legend(loc='best')
fig.savefig('MINE_window.png')
fig.show()



error=(abs(mv_av[-1]-mi)/mi)*100
print('true value:')
print(mi)
print('calculated value:')
print(mv_av[-1])
print('error')
print(error)
print('\n')




C:\Users\ahmedadel\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:17: RuntimeWarning: covariance is not symmetric positive-semidefinite.
C:\Users\ahmedadel\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:21: RuntimeWarning: covariance is not symmetric positive-semidefinite.


In [ ]:
def initialize_parameters(d):
    """
    Initializes weight parameters to build a neural network with tensorflow. The shapes are:
                        W1 : [4, 4, 3, 8]
                        W2 : [2, 2, 8, 16]
    Returns:
    parameters -- a dictionary of tensors containing W1, W2
    """
    
    tf.set_random_seed(1)                              # so that your "random" numbers match ours
        
    ### START CODE HERE ### (approx. 2 lines of code)
    W1 = tf.get_variable("W1", [4, 4, 4, d, 5], initializer=tf.contrib.layers.xavier_initializer(seed=0))
    W2 = tf.get_variable("W2", [2, 2, 2, 5, 5], initializer=tf.contrib.layers.xavier_initializer(seed=0))
    ### END CODE HERE ###

    parameters = {"W1": W1,
                  "W2": W2}
    
    return parameters

In [ ]:
H=20
n_epochs = 2000
data_size = 20000

def MINE_CNN(x_in, y_in):
    
    # shuffle and concatenate
    y_shuffle = tf.random_shuffle(y_in)
    x_conc = tf.concat([x_in, x_in], axis=0)
    y_conc = tf.concat([y_in, y_shuffle], axis=0)
    
    # propagate the forward pass
    layerx = layers.linear(x_conc, H)
    layery = layers.linear(y_conc, H)
    layer2 = tf.nn.relu(layerx + layery)
    output = layers.linear(layer2, 1)
    
    # split in T_xy and T_x_y predictions
    N_samples = tf.shape(x_in)[0]
    T_xy = output[:N_samples]
    T_x_y = output[N_samples:]
    # compute the negative loss (maximise loss == minimise -loss)
    neg_loss = -(tf.reduce_mean(T_xy, axis=0) - tf.math.log(tf.reduce_mean(tf.math.exp(T_x_y))))
    opt = tf.train.AdamOptimizer(learning_rate=0.01).minimize(neg_loss)

    return neg_loss, opt

# prepare the placeholders for inputs
x_in = tf.placeholder(tf.float32, [None, 1], name='x_in')
y_in = tf.placeholder(tf.float32, [None, 1], name='y_in')

# make the loss and optimisation graphs
neg_loss, opt = MINE(x_in, y_in)

# start the session
sess = tf.InteractiveSession()
sess.run(tf.global_variables_initializer())

# train
MIs = []
for epoch in range(n_epochs):
    
    # generate the data
    x_sample=gen_X()
    noise_sample=noise()
    Y=gen_Y(x_sample,noise_sample)
    X_1D=X.reshape(-1,1)
    Y_1D=Y.reshape(-1,1)
    print (epoch, end="\r")

    # perform the training step
    feed_dict = {x_in:X_1D, y_in:X_1D}
    _, neg_l = sess.run([opt, neg_loss], feed_dict=feed_dict)
    
    # save the loss
    MIs.append(-neg_l)

mv_av=ma(MIs, policy='weighted', beta=0.01)



fig, ax = plt.subplots()
ax.plot([0, len(MIs)], [mi,mi], label='True Mutual Information')
ax.plot(range(len(mv_av)), mv_av, label='Moving Average of MINE (window policy)')

ax.set_xlabel('training steps')
ax.legend(loc='best')
fig.savefig('MINE_window.png')
fig.show()



error=(abs(mv_av[-1]-mi)/mi)*100
print('true value:')
print(mi)
print('calculated value:')
print(mv_av[-1])
print('error')
print(error)
print('\n')




C:\Users\ahmedadel\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:17: RuntimeWarning: covariance is not symmetric positive-semidefinite.
C:\Users\ahmedadel\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:21: RuntimeWarning: covariance is not symmetric positive-semidefinite.
